<a href="https://colab.research.google.com/github/raposeidon/AI-education/blob/main/colabs/%EC%99%B8%EB%B6%80_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%A1%9C%EC%BB%AC_%ED%8C%8C%EC%9D%BC%2C_%EB%93%9C%EB%9D%BC%EC%9D%B4%EB%B8%8C%2C_%EC%8A%A4%ED%94%84%EB%A0%88%EB%93%9C%EC%8B%9C%ED%8A%B8%2C_Cloud_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 노트북에서는 외부 소스의 데이터를 로드 및 저장하는 레시피를 이용할 수 있습니다.

# 로컬 파일 시스템

## 로컬 파일 시스템의 파일 업로드

<code>files.upload</code>는 업로드된 파일의 사전을 반환합니다.
사전은 업로드된 파일 이름에 따라 키가 지정되며, 값은 업로드된 데이터를 표시합니다.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

## 로컬 파일 시스템으로 파일 다운로드

<code>files.download</code>는 파일의 브라우저 다운로드를 로컬 컴퓨터로 호출합니다.


In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

# Google Drive

다음과 같은 여러 가지 방법으로 드라이브에 있는 파일에 액세스할 수 있습니다.
- 런타임 가상 머신에 Google Drive 마운트
- <a href="https://pythonhosted.org/PyDrive/">PyDrive</a>와 같은 API에 래퍼 사용
- <a href="https://developers.google.com/drive/v3/web/about-sdk">네이티브 REST API</a> 사용



각각의 예시는 다음과 같습니다.

## 로컬에 Google Drive 마운트하기

아래의 예시에서는 승인 코드를 사용해 런타임에 Google Drive를 마운트하는 방법 및 이 드라이브에서 파일을 쓰고 읽는 방법을 보여 줍니다. 이 예시를 실행하면 <a href="https://drive.google.com/">https://drive.google.com/</a>에 새 파일&#40;<code>foo.txt</code>&#41;이 표시됩니다.

이 방법으로는 파일을 읽고, 쓰고 이동하는 작업만 가능하므로 프로그래매틱 방식으로 공유 설정이나 기타 메타데이터를 수정하려면 아래의 다른 옵션 중 하나를 사용하세요.

<strong>참고:</strong> 파일 브라우저의 '드라이브 마운트' 버튼을 사용할 때 현재 사용자만 수정한 메모장에 대해서는 인증 코드는 필요하지 않습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code
Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


## PyDrive

아래 예시에서는 PyDrive를 사용한 인증 및 파일 업로드/다운로드 방법을 확인할 수 있습니다. <a href="https://pythonhosted.org/PyDrive/">PyDrive 도움말</a>에서 더 많은 예시를 살펴보세요.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

PyDrive 클라이언트를 인증하고 만듭니다.


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

텍스트 파일을 만들고 업로드합니다.


In [ ]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 14vDAdqp7BSCQnoougmgylBexIr2AQx2T


ID별로 파일을 로드하고 파일 내용을 출력합니다.


In [ ]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## Drive REST API

Drive API를 사용하려면 인증한 다음 API 클라이언트를 구성해야 합니다.


In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

이 클라이언트를 사용하면 <a href="https://developers.google.com/drive/v3/reference/">Google Drive API 참조</a>에 있는 모든 기능을 사용할 수 있습니다. 기능의 예시는 다음과 같습니다.


### Python 데이터로 새 드라이브 파일 만들기

먼저 업로드할 로컬 파일을 만듭니다.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

<a href="https://developers.google.com/drive/v3/reference/files/create"><code>files.create</code></a> 방식을 사용해 업로드합니다. 업로드 파일에 관한 추가 정보는 <a href="https://developers.google.com/drive/v3/web/manage-uploads">개발자 도움말</a>에서 확인할 수 있습니다.

In [ ]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1Cw9CqiyU6zbXFD9ViPZu_3yX-sYF4W17


위의 셀을 실행하면 <a href="https://drive.google.com/">https://drive.google.com/</a>에 'Sample file'이라는 새 파일이 표시됩니다.

### 드라이브 파일 데이터를 Python으로 다운로드

위에서 업로드한 파일을 다운로드합니다.

In [ ]:
file_id = created.get('id')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

Downloaded file contents are: b'my sample file'


다른 파일을 다운로드하려면 위의 <code>file&#95;id</code>를 파일의 ID로 설정하세요. ID의 형식은 '1uBtlaggVyWshwcyP6kEI-y&#95;W3P8D26sz'와 같습니다.

# Google Sheets

아래 예시는 오픈소스 <a href="https://github.com/burnash/gspread"><code>gspread</code></a> 라이브러리를 사용해 Google Sheets와 상호작용합니다.

라이브러리를 가져오고 인증한 다음 스프레드시트에 인터페이스를 만듭니다.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

다음은 몇 가지 <code>gspread</code> 예시입니다. 추가 예시는 <a href="https://github.com/burnash/gspread#more-examples"><code>gspread</code> GitHub 페이지</a>에서 확인할 수 있습니다.

## Python 데이터로 새 시트 만들기

In [ ]:
sh = gc.create('My cool spreadsheet')

위의 셀을 실행하면 <a href="https://sheets.google.com/">https://sheets.google.com</a>에 'My cool spreadsheet'라는 새 스프레드시트가 표시됩니다.

새 시트를 열고 몇 가지 데이터를 무작위로 추가합니다.

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1dsQeN0YzXuM387l_CuyEbsYzL2ew9TJFzR-E-RQnwxs',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': 'Sheet1!A1:C2',
 'updatedRows': 2}

## 시트의 데이터를 Pandas DataFrame으로 Python에 다운로드

위에서 삽입한 무작위 데이터를 다시 읽고 결과를 <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html">Pandas DataFrame</a>으로 변환합니다.

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
pd.DataFrame.from_records(rows)

[['6', '3', '4'], ['7', '2', '1']]


,0,1,2
0,6,3,4
1,7,2,1


# Google Cloud Storage&#40;GCS&#41;

GCS와 함께 Colaboratory를 사용하려면 <a href="https://cloud.google.com/storage/docs/projects">Google Cloud 프로젝트</a>를 만들거나 기존 프로젝트를 사용해야 합니다.

아래에서 프로젝트 ID를 지정하세요.

In [ ]:
project_id = 'Your_project_ID_here'

GCS의 파일은 <a href="https://cloud.google.com/storage/docs/buckets">버킷</a>에 보관됩니다.

버킷에는 전역 고유 이름이 필요하므로 여기에서 하나 생성하겠습니다.

In [ ]:
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

GCS에 액세스하려면 인증이 필요합니다.

In [ ]:
from google.colab import auth
auth.authenticate_user()

GCS는 <code>gsutil</code> 명령줄 유틸리티 또는 네이티브 Python API를 통해 액세스할 수 있습니다.

## `gsutil`

먼저 <code>gcloud</code>를 사용해 위에서 지정한 프로젝트를 사용할 수 있도록 <code>gsutil</code>을 설정합니다.

In [ ]:
!gcloud config set project {project_id}

Updated property [core/project].


업로드할 로컬 파일을 만듭니다.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

파일을 업로드할 버킷을 만듭니다&#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/mb">도움말</a>&#41;.

In [ ]:
!gsutil mb gs://{bucket_name}

Creating gs://colab-sample-bucket-44971372-baaf-11e7-ae30-0242ac110002/...


파일을 새 버킷으로 복사합니다&#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cp">도움말</a>&#41;.

In [ ]:
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

Copying file:///tmp/to_upload.txt [Content-Type=text/plain]...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       


모든 항목이 제대로 작동하도록 새롭게 복사한 파일의 내용을 덤프합니다&#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cat">도움말</a>&#41;.


In [ ]:
!gsutil cat gs://{bucket_name}/to_upload.txt

my sample file

In [ ]:
# @markdown 업로드가 완료되면 프로젝트의 Cloud Console Storage 브라우저에 데이터가 표시됩니다.
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


마지막으로 위 예시에 방금 업로드한 파일을 다운로드해 보겠습니다. <code>gsutil cp</code> 명령어에서 순서를 반대로 바꾸기만 하면 됩니다.

In [ ]:
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt

# 전송이 제대로 작동하는지 확인하기 위해 결과를 출력합니다.
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucket483f20dc-baaf-11e7-ae30-0242ac110002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

## Python API

다음 스니펫은 API의 추가적인 사용 사례를 보여 주는 <a href="https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py">대규모 예시</a>를 기반으로 합니다.

먼저 서비스 클라이언트를 만듭니다.

In [ ]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

업로드할 로컬 파일을 만듭니다.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

위에서 지정된 프로젝트에 버킷을 만듭니다.

In [ ]:
# 위의 gsutil 예시와 다른 전역 고유 버킷 이름을 사용하세요.
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

Done


새롭게 만든 버킷에 파일을 업로드합니다.

In [ ]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name,
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

Upload complete


In [ ]:
# @markdown 업로드가 완료되면 프로젝트의 Cloud Console Storage 브라우저에 데이터가 표시됩니다.
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


방금 업로드한 파일을 다운로드합니다.

In [ ]:
from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Download complete


다운로드된 파일을 검사합니다.


In [ ]:
!cat /tmp/downloaded_from_gcs.txt

my sample file